# Introduction

Avant de commencer, parcourer le fichier README.rst

# Requête HTTP 

Un requête HTTP est une requête basé sur le protocole TCP, elle fait partie de la couche application de la couche OSI. Elle permet d'accéder aux données mise à disposition sur une adresse IP (ou url résolue par un DNS) et un port. 

Les deux ports les plus utilisé dans le web sont le 80 pour les sites en HTTP et le 443 pour les sites en HTTPS. HTTPS est une variable du protocole HTTP basé sur le protocole TLS.

Il existe de nombreux types de requêtes selon la convention `REST`: 
- GET
- POST
- PUT 
- DELETE
- UPDATE.

Dans notre cas nous allons utiliser la plupart du temps des GET et potentiellement des POST. 
- Le GET permet comme sont nom l'indique de récupérer des informations en fonction de certain paramètres. 
- Le POST nécéssite un envoie de données pour récupérer des données. Le body du post est, la plupart du temps, envoyé sous la forme d'un objet JSON.

Ces requêtes encapsulent un certain nombre de paramètres qui permettent soient d'identifier une provenance et un utilisateur ou de réaliser différentes actions.

In [1]:
import requests

In [2]:
url = "http://www.esiee.fr/"
response = requests.get(url)
response.status_code

200

Il existe deux méthodes pour récupérer le contenu de la page :

- `response.text` qui permet de retourner le texte sous la forme d'une chaine de charactères.
- `response.content` qui permet de récupérer le contenu de la page sous la forme de bytes

In [3]:
type(response.content)

bytes

In [4]:
type(response.text)

str

Pour récupérer les 1000 premiers charactères de la page :

In [5]:
response.text[0:1000]

'<!DOCTYPE html>\n<html lang="fr-FR">\n<head>\n\n<meta charset="utf-8">\n<!-- \n\tThis website is powered by TYPO3 - inspiring people to share!\n\tTYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL.\n\tTYPO3 is copyright 1998-2023 of Kasper Skaarhoj. Extensions are copyright of their respective owners.\n\tInformation and contribution at https://typo3.org/\n-->\n\n\n\n<title>ESIEE Paris, l&#039;école de l&#039;innovation technologique | ESIEE Paris</title>\n<meta name="generator" content="TYPO3 CMS" />\n<meta name="description" content="Rejoignez ESIEE Paris, grande école d&#039;ingénieur dans les domaines des transitions numérique, énergétique et environnementale. Classée dans le groupe A, parmi les meilleures écoles d&#039;ingénieur selon le classement de l&#039;Etudiant. Habilitée par la Commission des Titres d&#039;Ingénieur (CTI). Membre de la Conférence des Grandes Ecoles (CGE). " />\n<meta name="viewport" conte

Pour récupérer les headers HTTP de la réponse :

In [6]:
response.headers

{'Date': 'Tue, 14 Nov 2023 13:12:22 GMT', 'Server': 'Apache', 'Content-Language': 'fr', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip', 'X-UA-Compatible': 'IE=edge', 'X-Content-Type-Options': 'nosniff', 'Content-Length': '15453', 'Content-Type': 'text/html; charset=utf-8', 'X-Varnish': '17586308 4544289', 'Age': '95', 'Via': '1.1 varnish (Varnish/7.1)', 'Accept-Ranges': 'bytes', 'Connection': 'keep-alive'}

On peut modifier les paramêtres de la requête et/ou ses headers. On peut par exemple ajouter un UserAgent et un timeout de 10 secondes:

In [7]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get(url, headers=headers, timeout = 10)
response.content[0:1000]

b'<!DOCTYPE html>\n<html lang="fr-FR">\n<head>\n\n<meta charset="utf-8">\n<!-- \n\tThis website is powered by TYPO3 - inspiring people to share!\n\tTYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL.\n\tTYPO3 is copyright 1998-2023 of Kasper Skaarhoj. Extensions are copyright of their respective owners.\n\tInformation and contribution at https://typo3.org/\n-->\n\n\n\n<title>ESIEE Paris, l&#039;\xc3\xa9cole de l&#039;innovation technologique | ESIEE Paris</title>\n<meta name="generator" content="TYPO3 CMS" />\n<meta name="description" content="Rejoignez ESIEE Paris, grande \xc3\xa9cole d&#039;ing\xc3\xa9nieur dans les domaines des transitions num\xc3\xa9rique, \xc3\xa9nerg\xc3\xa9tique et environnementale. Class\xc3\xa9e dans le groupe A, parmi les meilleures \xc3\xa9coles d&#039;ing\xc3\xa9nieur selon le classement de l&#039;Etudiant. Habilit\xc3\xa9e par la Commission des Titres d&#039;Ing\xc3\xa9nieur (CTI). Membr

## Exercice

## Exercice 1

- Créer une classe Python permettant de faire des requêtes HTTP.
- Cette classe doit utiliser toujours le même UserAgent.
- Le TimeOut sera spécifié à chaque appelle avec une valeur par défaut.
- Un mécanisme de retry sera mis en place de façon recursive.


## Exercice 2

- Faire une fonction permettant de supprimer tous les espaces supperflus d'une string
- Faire une fonction qui prend une string html et renvois une string intelligible (enlever les caractères spéciaux,
- Récupérer le domaine en fonction d'un url

In [53]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse

class HTTPrequest:
    
    def __init__(self, timeOut = 10, retry = 3):
        self.UserAgent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0'
        self.timeOut = timeOut
        self.retry = retry
        self.response = None
        self.html_txt = ""
        self.txt = ""
        
    
    
    def make_request(self, url):
        try:
            self.response = requests.get(
                url,
                headers = {'User-Agent': self.UserAgent},
                timeout=self.timeOut,
            )
            self.response.raise_for_status()
            self.html_txt = self.response.text
        except requests.exceptions.RequestException as e:
            if self.retry > 0:
                print(f"Request failed. Retrying... ({retries} retries remaining)")
                self.retry -= 1
                return self.make_request(url)
            else:
                print("Maximum retries reached. Request failed.")
                raise e
                
    def remove_extra_spaces(self):
        return ' '.join(self.html_txt.split())
    
    def clean_html(self):
        soup = BeautifulSoup(self.html_txt, 'html.parser')
        text_content = soup.get_text(separator=' ')
        self.text =  re.sub(r'\s+', ' ', text_content).strip()
        return self.text
    
    def get_domain_from_url(self, url):
        parsed_url = urlparse(url)
        return parsed_url.netloc

        


url = "http://www.esiee.fr/"
http_requester = HTTPrequest()
response = http_requester.make_request(url)
print(http_requester.remove_extra_spaces()[:1000])
print("-"*180)
print(http_requester.clean_html()[:1000])
print("-"*180)
print(http_requester.get_domain_from_url(url))

<!DOCTYPE html> <html lang="fr-FR"> <head> <meta charset="utf-8"> <!-- This website is powered by TYPO3 - inspiring people to share! TYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL. TYPO3 is copyright 1998-2023 of Kasper Skaarhoj. Extensions are copyright of their respective owners. Information and contribution at https://typo3.org/ --> <title>ESIEE Paris, l&#039;école de l&#039;innovation technologique | ESIEE Paris</title> <meta name="generator" content="TYPO3 CMS" /> <meta name="description" content="Rejoignez ESIEE Paris, grande école d&#039;ingénieur dans les domaines des transitions numérique, énergétique et environnementale. Classée dans le groupe A, parmi les meilleures écoles d&#039;ingénieur selon le classement de l&#039;Etudiant. Habilitée par la Commission des Titres d&#039;Ingénieur (CTI). Membre de la Conférence des Grandes Ecoles (CGE). " /> <meta name="viewport" content="width=device-width, user-sc

# Exploitation du HTML  

Ici, il faut récupérer le code HTML d'un site web à partir d'une requête. Lorsque vous avez récupéré le texte d'un site il faut le parser. Pour cela, on utilise BeautifulSoup qui permet de transformer la structure HTML en objet Python. Cela permet de récupérer efficacement les données qui nous intéresse.

Pour les webmasters, le blocage le plus souvent mis en place et un blocage sur le User-Agent. Le User-Agent est un paramètre intégré dans la requête HTTP réalisé par le Navigateur pour envoyer au front des informations basiques :

- la version du Navigateur,
- la version de l'OS
- Le type de gestionnaire graphique (Gecko)
- le type de device utilisé

Exemple de User Agent :  

`Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0`

Commençons à utiliser `BeautifulSoup`, pour l'installer : 

In [8]:
!pip install bs4
!pip install  lxml

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1264 sha256=eff975962ee5027fdc2aa0a6eea0c9f1c3fa6ca6e627f1b047c12b6e9bad62d8
  Stored in directory: c:\users\adrie\appdata\local\pip\cache\wheels\d4\c8\5b\b5be9c20e5e4503d04a6eac8a3cd5c2393505c29f02bea0960
Successfully built bs4


In [63]:
import requests
from bs4 import BeautifulSoup

Pour transformer une requête (requests) en objet BeautifulSoup :

In [64]:
response = requests.get(url)
soup = BeautifulSoup(response.text)

Il se peut qu'un message d'erreur arrive à ce point là si vous n'avez pas la librarie `lxml` installée, pour se faire vous avez juste à lancer la commande suivante : 

In [65]:
!pip install lxml

Pour trouver tous les liens d'une page on récupère la balise `a` qui permet de gérer les liens en HTML  :

In [66]:
soup.find_all("a")[0:10]

[<a href="/#content">Aller au contenu</a>,
 <a href="/#menu">Aller au menu</a>,
 <a href="/plan-du-site/">Plan du site</a>,
 <a href="/nous-rencontrer-1">Bien choisir son école, c'est aussi la rencontrer  ! Rendez-vous à la prochaine journée portes ouvertes le 9 décembre.</a>,
 <a href="/"><img alt="ESIEE PARIS" class="a42-ac-replace-img" src="/typo3conf/ext/esiee_sitepackage/Resources/Public/imgs/svg/logo-esiee.svg"/></a>,
 <a href="/brochures-1">Brochures</a>,
 <a href="/informations/etudiantes-et-etudiants">Espace élèves</a>,
 <a href="/" hreflang="fr-FR" title="Français">
 <span>Fr</span>
 </a>,
 <a href="/en/" hreflang="en-US" title="English">
 <span>En</span>
 </a>,
 <a href="/candidater-1">Candidater</a>]

On peut préciser la classe HTML voulue  pour l'ensemble des `a`:

```python
soup.find_all(class_="<CLASS_NAME>")[0:10]
```

Ici par exemple: 

In [67]:
soup.find_all(class_="slide")[0:5]

[]

Pour récupérer le text sans les balises HTML :

In [68]:
soup.text[0:1000]

"\n\n\n\nESIEE Paris, l'école de l'innovation technologique | ESIEE Paris\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAller au contenu\nAller au menu\nPlan du site\n\n\n\n\n\n\n\nBien choisir son école, c'est aussi la rencontrer  ! Rendez-vous à la prochaine journée portes ouvertes le 9 décembre.\n\n\n\n\n\nMasquer l'alerte\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBrochuresEspace élèves\n\n\n\nFr\n\n\n\n\nEn\n\n\n\n\n\n\n\n\n\nAfficherMasquer la recherche\r\n\t\t\n\n\n\nSaisissez votre recherche\xa0:\n\nLancer la recherche\n\n\n\nCandidater\n\nAfficherMasquer le menu\n\n\n\n\n\nRetour au menu principalAfficherMasquer le sous menu\xa0: L'écolePourquoi choisir ESIEE Paris ?AfficherMasquer le sous menu\xa0: Gouvernance et conseilsGouvernance et conseilsAfficherMasquer le sous menu\xa0: Départements d'enseignements et de rechercheInformatique et télécommunicationsIngénierie des systèmes cyberphysiquesIngénierie industrielleSanté, énergie et environnement durableManagement, sciences humaines et la

## Exercice
### Exercice 3

Améliorer la classe développé précédemment.

- Ajouter une méthode pour récupérer l'objet soup d'un url
- Récupérer une liste de User Agent et effectuer une rotation aléatoire sur celui à utiliser
- Utiliser cette classe pour parser une page HTML et récupérer : le titre, tous les H1 (si ils existent), les liens vers les images, les liens sortants vers d'autres sites, et le texte principal.

Parsing d'un sitemaps pour récupérer une listes de liens avec les informations disponibles. -> Stocker dans un dictionnaire et dans un fichier JSON local.

In [145]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse
import random
import json

class HTTPrequest:
    
    def __init__(self, timeOut=10, retry=3):
        self.UserAgents = ['Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0',
                           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36']
        self.UserAgent = ''
        self.timeOut = timeOut
        self.retry = retry
        self.response = None
        self.html_txt = ""
        self.txt = ""
        self.url = ""
        self.soup = None
        self.title = ""
        self.H1 = []
        self.image = []
        self.link = []
        self.main_txt = ""
        
    def make_request(self, url):
        try:
            self.response = requests.get(
                url,
                headers={'User-Agent': random.choice(self.UserAgents)},
                timeout=self.timeOut,
            )
            self.response.raise_for_status()
            self.html_txt = self.response.text
        except requests.exceptions.RequestException as e:
            if self.retry > 0:
                print(f"Request failed. Retrying... ({self.retry} retries remaining)")
                self.retry -= 1
                return self.make_request(url)
            else:
                print("Maximum retries reached. Request failed.")
                raise e
    
    def remove_extra_spaces(self):
        return ' '.join(self.html_txt.split())
    
    def clean_html(self):
        self.soup = BeautifulSoup(self.html_txt, 'html.parser')
        text_content = self.soup.get_text(separator=' ')
        self.text = re.sub(r'\s+', ' ', text_content).strip()
        return self.text
    
    def get_domain_from_url(self, url):
        parsed_url = urlparse(url)
        self.url = parsed_url.netloc
        return self.url
    
    def get_soup(self):
        return self.soup
    
    def extract_title(self):
        title_tag = self.soup.find_all('title')
        if title_tag:
            self.title = title_tag
        else:
            self.title = "No title found"
            
    def extract_h1(self):
        title_tag = self.soup.find_all('h1')
        if title_tag:
            self.H1 = title_tag
        else:
            self.H1 = "No H1 found"
            
    def extract_img(self):
        title_tag = self.soup.find_all('img')
        if title_tag:
            self.image = title_tag
        else:
            self.image = "No image found"
            
    def extract_link(self):
        title_tag = self.soup.find_all('a')
        if title_tag:
            self.link = title_tag
        else:
            self.link = "No link found"
            
    def extract_main(self):
        title_tag = self.soup.find_all('p')
        if title_tag:
            self.main_txt = title_tag
        else:
            self.main_txt = "No main text found"
            
    def parse_sitemap(self):
        sitemap_data = []
        a = {}
        if self.response:
            a[f"{self.url}"] = {
            "title": self.title,
            "h1_tags": self.H1,
            "image_links": self.image,
            "external_links": self.link,
            "main_text": self.main_txt
            }
            sitemap_data.append( a)
            return sitemap_data
            
    def save_to_json(self, data, filename='output.json'):
        with open(filename, 'w') as json_file:
            return json_file.write(str(data))

url = "http://www.esiee.fr/"
http_requester = HTTPrequest()
http_requester.make_request(url)
print(http_requester.get_domain_from_url(url))
print("-"*180)
print(http_requester.response.text[:1000])
print("-"*180)
print(http_requester.remove_extra_spaces()[:1000])
print("-"*180)
print(http_requester.clean_html()[:1000])
print("-"*180)
print(http_requester.get_soup().text[:100])
print("-"*180)
http_requester.extract_title()
print(f"titre: {http_requester.title}")
print("-"*180)
http_requester.extract_h1()
print(f"H1: {http_requester.H1}")
print("-"*180)
http_requester.extract_img()
print(f"image: {http_requester.image[0:5]}")
print("-"*180)
http_requester.extract_link()
print(f"link: {http_requester.link[0:5]}")
print("-"*180)
http_requester.extract_main()
print(f"main text: {http_requester.main_txt[0:10]}")
f = http_requester.parse_sitemap()
http_requester.save_to_json(f)

www.esiee.fr
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
<!DOCTYPE html>
<html lang="fr-FR">
<head>

<meta charset="utf-8">
<!-- 
	This website is powered by TYPO3 - inspiring people to share!
	TYPO3 is a free open source Content Management Framework initially created by Kasper Skaarhoj and licensed under GNU/GPL.
	TYPO3 is copyright 1998-2023 of Kasper Skaarhoj. Extensions are copyright of their respective owners.
	Information and contribution at https://typo3.org/
-->



<title>ESIEE Paris, l&#039;école de l&#039;innovation technologique | ESIEE Paris</title>
<meta name="generator" content="TYPO3 CMS" />
<meta name="description" content="Rejoignez ESIEE Paris, grande école d&#039;ingénieur dans les domaines des transitions numérique, énergétique et environnementale. Classée dans le groupe A, parmi les meilleures écoles d&#039;ingénieur selon le clas

59408

# Exploitation des appels d'API



Losque le front du site récupère des données sur une API géré par le back, un appel d'API est réalisé. Cet appel est recensé dans les appels réseaux. Il est alors possible de re-jouer cet appel pour récupérer à nouveau les données. Il est très facile de récupérer ces appels dans l'onglet Network de la console développeur de Chrome ou FireFox. La console vous permet de copier le code CURL pour effectuée et vous pouvez ensuite la transformer en code Python depuis le site https://curl.trillworks.com/.

Souvent les APIs sont bloquées avec certain paramètres. L'API verifie que dans les headers de la requêtes HTTP ces paramètres sont présents : * un token généré à la volée avec des protocole OAuth2 (ou moins développés). * un referer provenant du site web (la source de la requête), très facile à falsifier.



## Exercice 
### Exercice 4

- Utiliser les informations développées plus haut pour récupérer les premiers résultats d'une recherche d'une requête
sur Qwant. 

Tips : 

- Aller sur https://www.qwant.com/
- Ouvrir les outils de développements de Chrome ou Firefox
- Onglet Network
- Fouiller dans les requêtes

# Exercice Final  

Exercice Final
Utilisez tout ce que vous avez appris pour récupérer des articles de News avec une catégorie. Il est souvent intéressant de partir des flux RSS pour commencer :

Les données doivent comprendre :
- Le texte important propre
- L'url
- Le domaine
- la catégorie
- Le titre de l'article
- Le titre de la page
- (Facultatif) : les images

Tips : 

- Taper le nom de votre média favoris + RSS (par exemple : https://www.lemonde.fr/rss/)
- Aller dans le DOM de la page 
- Trouver les catégories et les liens vers les articles